In [35]:
import pandas as pd
# import spacy
import pickle
import os
import random
import numpy as np
from numpy.linalg import norm
from datetime import datetime

os.getcwd()

'd:\\OneDrive - Universiteit Utrecht\\PER3_PersonalizationForPublicMedia\\assignment2\\General\\Code'

In [36]:
with open("data/articles_cleaned.pkl", "rb") as file:
    articles = pickle.load(file)

articles_topics = pd.read_csv('data/articles_topics.csv')
articles_topics = articles_topics.loc[:, articles_topics.columns != 'Unnamed: 0']

personas = pd.read_csv('data/personas.csv')

In [37]:
def calculate_articlesInterest(persona):
    #Calculate interest score for every article
    #needs topic df and persona vector
    #Outputs series objects filled with scores for every article
    
    #Transform into arrays
    articles_matrix = articles_topics.to_numpy()

    #Only use cols that correspond to interests
    interest_cols = persona.index[persona.index.str.startswith('interest_')]
    user_vector = np.array(persona[interest_cols])
    
    #Calculate new scores using matrix multiplication
    new_scores_matrix = np.dot(articles_matrix, user_vector)
    new_scores_series = pd.Series(new_scores_matrix)

    return new_scores_series

In [38]:
def calculate_articlesDistressed(persona):
    #Calculate distressed score for every article
    #needs topic df and persona vector
    #Outputs series objects filled with scores for every article
    
    #Transform into arrays
    articles_matrix = articles_topics.to_numpy()

    #Only use cols that correspond to distressed
    distressed_cols = persona.index[persona.index.str.startswith('distressed_')]
    user_vector = np.array(persona[distressed_cols])
    
    #Calculate new scores using matrix multiplication
    new_scores_matrix = np.dot(articles_matrix, user_vector)
    new_scores_series = pd.Series(new_scores_matrix)

    return 1-new_scores_series


In [39]:
def newPersona(persona_idx, weights = [0.8, 0.2]):
    allCols = ['interest_EUPolitics', 'interest_crimes', 'interest_israelPalestine', 'interest_immigration', 'interest_sports', 'interest_war', 'interest_climateChange', 'interest_showArts', 'interest_covid', 'interest_britishBrexit', 'interest_instAbuse', 'interest_spaceTravel', 'interest_protests', 'interest_terrorism', 'interest_USPolitics', 'interest_naturalDisasters', 'interest_elections', 'interest_economy', 'distressed_EUPolitics', 'distressed_crimes', 'distressed_israelPalestine', 'distressed_immigration', 'distressed_sports', 'distressed_war', 'distressed_climateChange', 'distressed_showArts', 'distressed_covid', 'distressed_britishBrexit', 'distressed_instAbuse', 'distressed_spaceTravel', 'distressed_protest', 'distressed_terrorism', 'distressed_USPolitics', 'distressed_naturalDisasters', 'distressed_elections', 'distressed_economy', 'guard_suicide', 'guard_accidents', 'guard_selfHarm', 'guard_depression', 'guard_racism', 'guard_eatingDisorders']
    persona = pd.Series(index = allCols)

    interest_cols = persona.index[persona.index.str.startswith('interest_')]
    distressed_cols = persona.index[persona.index.str.startswith('distressed_')]
    guard_cols = persona.index[persona.index.str.startswith('guard_')]

    # initialise data from survey means
    persona[interest_cols] = personas.iloc[persona_idx][interest_cols]
    persona[distressed_cols] = personas.iloc[persona_idx][distressed_cols]/100
    persona[guard_cols] = personas.iloc[persona_idx][guard_cols]

    # create article scores
    article_scores = pd.DataFrame(0, index = articles.index, columns = ['interest', 'distress', 'total'])

    article_scores['interest'] = calculate_articlesInterest(persona)
    article_scores['distress'] = calculate_articlesDistressed(persona)

    scores_weighted = weights * article_scores.iloc[:,:len(weights)]
    article_scores['total'] = scores_weighted.apply(lambda row : row.sum(), axis=1)

    return persona, article_scores

persona0_series, article_scores_persona0 = newPersona(0)
persona1_series, article_scores_persona1 = newPersona(1)
persona2_series, article_scores_persona2 = newPersona(2)
persona3_series, article_scores_persona3 = newPersona(3)
persona4_series, article_scores_persona4 = newPersona(4)
# print(persona)
# article_scores

C:\Users\maike\AppData\Local\Temp\ipykernel_9256\2982752710.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  persona = pd.Series(index = allCols)
C:\Users\maike\AppData\Local\Temp\ipykernel_9256\2982752710.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  persona = pd.Series(index = allCols)
C:\Users\maike\AppData\Local\Temp\ipykernel_9256\2982752710.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  persona = pd.Series(index = allCols)
C:\Users\maike\AppData\Local\Temp\ipykernel_9256\2982752710.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly 

In [40]:
article_scores_personas = pd.DataFrame(index = articles.index)
article_scores_personas['persona0'] = article_scores_persona0['total']
article_scores_personas['persona1'] = article_scores_persona1['total']
article_scores_personas['persona2'] = article_scores_persona2['total']
article_scores_personas['persona3'] = article_scores_persona3['total']
article_scores_personas['persona4'] = article_scores_persona4['total']

In [41]:
article_scores_personas.to_csv('data/article_scores_personas.csv')